In [12]:
from stockdex import Ticker
import methods.stockdex_patch as stockdex_patch

In [19]:
data = Ticker("KO").macrotrends_key_financial_ratios
display(data)

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31
field_name,,,,,,,,,,,,,,
Current Ratio,1.02960,1.13410,1.14540,1.13010,1.31770,0.75670,0.86620,1.34390,1.28180,1.24010,1.01890,1.12560,1.09010,1.05000
Long-term Debt / Capital,0.61640,0.56400,0.58480,0.60520,0.65340,0.56600,0.57110,0.62170,0.56110,0.52360,0.38410,0.36420,0.30760,0.29960
Debt/Equity Ratio,1.68820,1.53070,1.51590,1.72010,2.01060,2.02690,2.32000,2.51280,1.96850,1.71230,1.36600,1.10880,0.98320,0.89500
Gross Margin,61.06330,59.52270,58.14340,60.27160,59.31120,60.77120,61.90380,62.10930,60.66930,60.53190,61.10920,60.68430,60.32030,60.86330
Operating Margin,21.23200,24.72130,25.36740,26.66670,27.25210,27.06490,26.68220,21.41560,20.67940,19.70470,21.10530,21.82950,22.44830,21.85770
EBIT Margin,21.23200,24.72130,25.36740,26.66670,27.25210,27.06490,26.68220,21.41560,20.67940,19.70470,21.10530,21.82950,22.44830,21.85770
EBITDA Margin,23.51630,27.18670,28.29740,30.42300,31.90470,30.72770,29.84840,24.89510,24.94800,24.15230,25.40110,26.04900,26.57600,26.05600
Pre-Tax Profit Margin,27.80650,28.30790,27.17420,32.14330,29.52990,28.94330,23.97960,19.02680,19.43480,21.68470,20.27260,24.49520,24.59340,24.61860
Net Profit Margin,22.58980,23.41650,22.18860,25.27750,23.46580,23.93600,18.75800,3.44640,15.59130,16.59590,15.43110,18.32070,18.78290,18.44360
